# JAX的即时编译

> 作者：Rosalia Schneider & Vladimir Mikulik

在本节中，我们将进一步讨论JAX的工作原理，以及如何使其具有高性能。我们将讨论`jax.jit()`变换，该变换将执行JAX Python函数的即时编译（JIT），以便可以在XLA中有效地执行该转换。

## 如何使用JAX变换

在上一节中，我们讨论了JAX允许我们变换Python函数。这是通过首先将Python函数转换为一种简单的中间语言jaxpr来完成的。之后，转换将在jaxpr形式上进行。

我们可以用 `jax.make_jaxpr` 来显示函数的jaxpr形式：

In [1]:
import jax
import jax.numpy as jnp

global_list = []

def log2(x):
    global_list.append(x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    
    return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0))

{ lambda  ; a.
  let b = log a
      c = log 2.0
      d = div b c
  in (d,) }


在教程中的[理解Jaxprs]()部分提供了有关上述输出含义的更多信息。

请注意，很重要的一点是jaxpr无法捕获该函数的副作用：其中没有与`global_list.append(x)`的内容。这是一个特性，并不是一个漏洞：JAX旨在理解无副作用的代码。如果您不太熟悉纯函数和副作用这两个术语，请参见[JAX锋芒毕露:🔪纯函数](https://render.githubusercontent.com/view/ipynb?color_mode=light&commit=fe4a5f85bf7936468ed39f20cced5b25a1612efb&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f726173696e2d7473756b7562612f4a41585f6368696e6573655f7475746f7269616c2f666534613566383562663739333634363865643339663230636365643562323561313631326566622f6f6666696369616c2d7475746f7269616c732f47657474696e67537461727465642f312e332d4a41582545392539342538422545382538412539322545362541462539352545392539432542322e6970796e62&nwo=rasin-tsukuba%2FJAX_chinese_tutorial&path=official-tutorials%2FGettingStarted%2F1.3-JAX%E9%94%8B%E8%8A%92%E6%AF%95%E9%9C%B2.ipynb&repository_id=349726397&repository_type=Repository#%F0%9F%94%AA%E7%BA%AF%E5%87%BD%E6%95%B0)。

当然，非纯函数仍然可以编写甚至运行，但是一旦转换为jaxpr，JAX就无法保证其行为。但根据经验，您可以期望（但不应该依赖）JAX转换函数的副作用只运行一次（在第一次调用时）之后再也不会运行。这是因为JAX使用称为“跟踪”的过程生成jaxpr的方式。

跟踪时，JAX用跟踪器对象包装每个参数。然后，这些跟踪器记录函数调用期间对他们执行的所有JAX操作（发生在Python代码之中）。之后，JAX使用跟踪记录来重构整个函数。该重建的输出是jaxpr。由于跟踪其没有记录Python的副作用，因此它们不会出现在jaxpr中。但是，副作用仍会在跟踪期间发生。

注意：Python的 `print()` 不是纯函数：文本输出是该函数的副作用。因此，任何 `print()`调用都只会在跟踪过程中发生，而不会出现在jaxpr中：

In [2]:
def log2_with_print(x):
    print("printed x: ", x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2)
    return ln_x / ln_2

print(jax.make_jaxpr(log2_with_print)(3.))

printed x:  Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
{ lambda  ; a.
  let b = log a
      c = log 2.0
      d = convert_element_type[ new_dtype=float32
                                weak_type=False ] b
      e = div d c
  in (e,) }


看到打印的`x`成为一个 `Traced` 对象了吗？这就是JAX的内部运行机制。

Python代码至少运行一次的事实严格上来说是实现细节，因此不应该对其有依赖。但是，理解它很有用，因为您可以调试以打印出计算的中间值时使用它。

关键要理解的是，jaxpr会捕获对给定参数执行的功能。例如，如果我们有条件，那么jaxpr将只知道我们采取的分支：

In [3]:
def log2_if_rank_2(x):
    if x.ndim == 2:
        ln_x = jnp.log(x)
        ln_2 = jnp.log(2)
        return ln_x / ln_2
    else:
        return x
    
print(jax.make_jaxpr(log2_if_rank_2)(jax.numpy.array([1., 2., 3.])))

{ lambda  ; a.
  let 
  in (a,) }


## 使用JIT编译函数

如前所述，JAX使操作可以使用相同的的代码在CPU、GPU和TPU上执行。让我们来看一个计算比例指数线性单位（SELU）的示例，这是深度学习中常用的一种运算：

In [4]:
import jax
import jax.numpy as jnp

def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()

852 µs ± 26 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


以上代码一次性向加速器发送了一个操作。这限制了XLA编译器优化功能的能力。

自然，我们想要做的事给XLA编译器尽可能多的代码，以便它可以完全优化它。为此，JAX提供了 `jax.jit`转换，它将即时编译JAX兼容的函数。下面的示例显示了如何使用JIT来加快此函数：

In [5]:
selu_jit = jax.jit(selu)

# warm up
#预热
selu_jit(x).block_until_ready()

%timeit selu_jit(x).block_until_ready()

55.5 µs ± 3.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


以下是刚才发生事情的详细解释：

1. 我们将`selu_jit`定义为`selu`的编译版本
2. 我们在`x`上运行一次 `selu_jit`。这就是JAX进行跟踪的地方——毕竟他需要一些输入才能包装在跟踪器中。然后，使用XLA将jaxpr编译为针对您的GPU或TPU优化的非常有效的代码。现在，对`selu_jit`的后续调用将使用改代码，从而完全跳过我们以前的Python实现。

（如果我们不单独包括预热调用，一切都会照常进行。它仍然会很快，因为我们在基准测试中运行了许多循环，但可能并不会公平比较。）

我们队编译版本的执行速度进行计时。（注意，由于JAX的异步执行模型，因此必须使用`block_until_ready()`）

## 为什么不都用上JIT？

看完以上的示例后，您可能想知道我们是否应该简单粗暴的将`jax.jit`应用于每个函数。要了解为什么不这么做，以及什么时候应该或不应该应用`jit`，首先让我们查看一下JIT无法正常工作的情况：

In [6]:
# Condition on value of x
# 以x的值为条件时

def f(x):
    if x> 0:
        return x
    else:
        return 2 * x
    
f_jit = jax.jit(f)
f_jit(10) # 应该会报错

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function f at <ipython-input-6-b9f850e0f0db>:4, this concrete value was not available in Python because it depends on the value of the arguments to f at <ipython-input-6-b9f850e0f0db>:4 at flattened positions [0], and the computation of these values is being staged out (that is, delayed rather than executed eagerly).
 (https://jax.readthedocs.io/en/latest/errors.html#jax._src.errors.ConcretizationTypeError)

In [7]:
# while loop conditioned on x and n
# while循环中以x和n值为条件

def g(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i

g_jit = jax.jit(g)
g_jit(10, 20) # 应该会报错

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function g at <ipython-input-7-72280d7a1183>:4, this concrete value was not available in Python because it depends on the value of the arguments to g at <ipython-input-7-72280d7a1183>:4 at flattened positions [1], and the computation of these values is being staged out (that is, delayed rather than executed eagerly).
 (https://jax.readthedocs.io/en/latest/errors.html#jax._src.errors.ConcretizationTypeError)

问题在于，我们试图将即时编译函数的输入值作为条件。我们无法执行此操作的原因与上述事实有关，jaxpr取决于用于跟踪它的实际值。

有关在跟踪中使用的值的信息越具体，我们将越可以使用标准的Python控制流来表达自己。但是，过于具体意味着我们无法将相同的跟踪函数用于其他值。JAX通过针对不同母的在不同的抽象级别进行跟踪来解决此问题。

对于`jax.jit`，默认几位别`ShapedArray`——也就是说，每个跟踪器都有具体的形状（虽然允许我们对其进行调整），但没有具体的值。这使得编译后的函数可以再所有可能具有相同形状的输入上工作，这是机器学习中的标准用例。但是，由于追踪器没有具体的值，因此如果我们尝试给一个跟踪器限定条件，则会得到上面的错误。

在`jax.grad`中，约束则更加宽松，您可以做更多的尝试。但是，如果要组合多个转换，则必须满足最严格的转换约束。因此，如果您在使用`jit(grad(f))`是，则`f`不能是一个限制条件。有关Python控制流和JAX交互之间的更多详细信息，请参见[JAX锋芒毕露:🔪 控制流](https://render.githubusercontent.com/view/ipynb?color_mode=light&commit=fe4a5f85bf7936468ed39f20cced5b25a1612efb&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f726173696e2d7473756b7562612f4a41585f6368696e6573655f7475746f7269616c2f666534613566383562663739333634363865643339663230636365643562323561313631326566622f6f6666696369616c2d7475746f7269616c732f47657474696e67537461727465642f312e332d4a41582545392539342538422545382538412539322545362541462539352545392539432542322e6970796e62&nwo=rasin-tsukuba%2FJAX_chinese_tutorial&path=official-tutorials%2FGettingStarted%2F1.3-JAX%E9%94%8B%E8%8A%92%E6%AF%95%E9%9C%B2.ipynb&repository_id=349726397&repository_type=Repository#%F0%9F%94%AA-%E6%8E%A7%E5%88%B6%E6%B5%81)。

解决问题的一种方法是重写代码，以免出现条件限制。另一个方法是使用特殊的控制流运算符，例如`jax.lax.cond`。但是，有时候也是不太可能的。在这种情况下，您可以考虑仅添加函数的一部分。例如，如果函数中计算量最大的部分位于循环内部，则我们可以仅对该内部部分进行JIT（请确保下一节有关缓存的内容，避免陷入混乱）：

In [8]:
# while loop conditioned on x and n with a jitted body
# 即时编译的while循环中以x和n为条件
@jax.jit
def loop_body(prev_i):
    return prev_i + 1

def g_inner_jitted(x, n):
    i = 0
    while i < n:
        i = loop_body(i)
    return x + i

g_inner_jitted(10, 20)

DeviceArray(30, dtype=int32)

如果我们确实要对一个对输入值有条件的函数进行JIT，我们可以通过制定`static_argnums`高速度JAX来帮助自己针对特定输入时使用不太抽象的跟踪器。这样做的代价是生成的jaxpr灵活性较差，因此JAX将不得不为指定输入的每个新值重新编译该函数。晋档保证该函数获得有限的不同值时，这才是一个好策略。

In [9]:
f_jit_correct = jax.jit(f, static_argnums=0)
print(f_jit_correct(10))

10


In [10]:
g_jit_correct = jax.jit(g, static_argnums=1)
print(g_jit_correct(10, 20))

30


## 使用JIT的时机

在许多以上示例都是不值得即时编译的：

In [11]:
print("g jitted:")
%timeit g_jit_correct(10, 20).block_until_ready()

print("g:")
%timeit g(10, 20)

g jitted:
51.8 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
g:
591 ns ± 4.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


这是因为`jax.jit`本身引入了一些开销。因此，荣昌只有在编译函数很复杂，并且要多次调用的情况下才用于节省时间。幸运的是，机器学习中经常有这种情况，我们倾向于编译一个大型、复杂的模型，然后运行数百万次迭代。

通常，您将最大的计算模块即时编译；理想的情况下，将整个`update`函数即时编译。这都为便以其提供了最大的优化自由度。

## 缓存

了解`jax.jit`的缓存行为很重要。

假设我们定义了`f=jax.jit(g)`。当我们第一次调用`f`时，它将被编译并缓存生成的XLA代码。`f`的后续调用将重用缓存的代码。`jax.jit`就是通过这种方式来弥补编译的前期成本。

如果我指定`static_argnums`，则缓存的代码将仅用于标记为`static`的相同参数值。如果其中任何一个发生更改，则会重新编译。如果有很多值，那么您的程序可能要花更多的时间在编译操作上，而不是执行。

避免在循环内调用`jax.jit`。这样做在每次调用时创建一个新的`f`，该`f`将在每次调用时进行编译，而不是重复使用相同的缓存函数：

In [12]:
def unjitted_loop_body(prev_i):
    return prev_i + 1

def g_inner_jitted_poorly(x, n):
    i = 0
    while i < n:
        # Don't do this
        # 别这么做
        i = jax.jit(unjitted_loop_body)(i)
    return x + i

print("jit called outside the loop:")
%timeit g_inner_jitted(10, 20).block_until_ready()

print("jit called inside the loop:")
%timeit g_inner_jitted_poorly(10, 20).block_until_ready()      

jit called outside the loop:
5.13 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
jit called inside the loop:
7.46 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
